#### Import Libraries

In [1]:
# Import required libraries for execution
import pandas as pd
import datetime
import os
import numpy as np

# Get username of the windows Account to be used in the folder paths
user = os.getenv('username')

#### Define Report Folders and Variables

In [2]:
# Define the folders for input files and final reports

main_folder = "C:/Users/"+user+"/Desktop/Airtel/Source Files/"
report_folder = "C:/Users/"+user+"/Desktop/Airtel/Reports/Active TD Report/"
report_folder_2 = "C:/Users/"+user+"/Desktop/Airtel/Reports/Active TD Report/Historical/"
mypath = "C:/Users/"+user+"/Desktop/Airtel/Airtel Reports/PD Sheets"

# Dates for file name and invoices to be read

curr_mon = datetime.datetime.today().strftime('%b%y') #'01Feb18'
curr_date = datetime.datetime.today().strftime('%d%b%y') #'01Feb18'

# Create monthly folders if not present already

if not os.path.exists(report_folder_2+"/"+curr_mon):
    os.makedirs(report_folder_2+"/"+curr_mon)
    
pd_file_name = 'PD - 29Jan19'
td_file_name = 'TD - 29Jan19'

emi_analysis_date = datetime.datetime.now().date() #'2018-12-14' Execution date in the format 'YYYY-MM-DD'

#### Import Files

In [3]:
# Excel File with Active/TD/PD details

xls = pd.ExcelFile(main_folder + pd_file_name + '.xlsx')

try:
    pd_data = pd.read_excel(xls, 'PD Base')
except:
    pd_name = input("Please enter the name of the PD sheet (Airtel changed it again, why am I not suprised?): ")
    pd_data = pd.read_excel(xls, pd_name)

xls = pd.ExcelFile(main_folder + td_file_name + '.xlsx')

try:
    td_data = pd.read_excel(xls, 'Details')
except:
    td_name = input("Please enter the name of the Active/TD sheet (Airtel changed it again, why am I not suprised?): ")
    td_data = pd.read_excel(xls, td_name)
    
adf_master = pd.read_csv(main_folder + 'master_anuj.csv')
borr_mobile = pd.read_csv(main_folder + 'borr_mobile.csv').fillna(0)
CLS_EMI_data = pd.read_csv(main_folder + 'CLS_EMI_Data.csv').fillna(0)

print("Data Reading Complete!")

Data Reading Complete!


### Check: Match PD Numbers with Billing details sheet

In [6]:
xls = pd.ExcelFile(main_folder + pd_file_name + '.xlsx')
try:
    pd_data_billing = pd.read_excel(xls, sheet_name='Billing Details')
except:
    pd_name = input("Please enter the name of the PD sheet (Airtel changed it again, why am I not suprised?): ")
    pd_data_billing = pd.read_excel(xls, sheet_name=pd_name)

pd_data_numbers = pd_data[['MOBILE_NO']].drop_duplicates(subset='MOBILE_NO')
pd_data_billing_numbers = pd_data_billing[['MOBILE_NO']].drop_duplicates(subset='MOBILE_NO')
pd_data_billing_numbers['flag'] = "Match"

pd_number_check = pd.merge(pd_data_numbers, pd_data_billing_numbers, on='MOBILE_NO', how='left').fillna("Not Match")

print(pd_number_check['flag'].value_counts())
print("if Not Match found please report to Airtel")

Match        2809
Not Match     213
Name: flag, dtype: int64
if Not Match found please report to Airtel


#### List Of Cases from Airtel

In [7]:
td_mobiles = list(td_data['Loan ID'].unique())
td_numbers = pd.DataFrame(data = td_mobiles, columns = ['loan_ref_no'])
td_numbers['Airtel_Status'] = 'Active/TD'

pd_mobiles = list(pd_data['Loan ID '].unique())
pd_numbers = pd.DataFrame(data = pd_mobiles, columns = ['loan_ref_no'])
pd_numbers['Airtel_Status'] = 'PD'

report_numbers = td_numbers.append(pd_numbers)

report_numbers['presence_flag'] = 1

report_numbers.shape

(26354, 3)

### Check 1: Missing Numbers from the sheet

In [8]:
borr_mobile = borr_mobile[(borr_mobile['Loan Application Status'] != 'Application Incomplete')]
borr_mobile = borr_mobile[(borr_mobile['Loan Application Status'] != 'Loan Cancelled')]
borr_mobile = borr_mobile[(borr_mobile['Loan Application Status'] != 'Seynse Approved')]

adf_master = adf_master[(adf_master['Loan Application Status'] != 'Application Incomplete')]
adf_master = adf_master[(adf_master['Loan Application Status'] != 'Loan Cancelled')]
adf_master = adf_master[(adf_master['Loan Application Status'] != 'Seynse Approved')]

adf_master = adf_master[(adf_master['Loan Application Status'] == 'Disbursed') & (adf_master['Is Employee'] == 0)]
borr_mobile = borr_mobile[(borr_mobile['Loan Application Status'] == 'Disbursed')]

borr_mobile = borr_mobile[['Loan Reference Number','MSISDN']]
borr_mobile = borr_mobile.rename(columns={'Loan Reference Number': 'loan_ref_no','MSISDN': 'Mobile_No'})
adf_master = adf_master.rename(columns={'Loan App Reference Number': 'loan_ref_no'})
# print(borr_mobile['Loan Application Status'].unique())

def to_int_conv(num):
    return int(num.replace('?',''))

borr_mobile['Mobile_No'] = borr_mobile.apply(lambda x: to_int_conv(str(x['Mobile_No'])),axis=1)
adf_master_msisdn = pd.merge(adf_master, borr_mobile, on = 'loan_ref_no', how = 'left')
delivered_devices = adf_master_msisdn#[adf_master_msisdn['delivery_flag']==1]

delivered_airtel_data = pd.merge(delivered_devices, report_numbers, on = 'loan_ref_no', how = 'left').fillna(0)
missing_data = delivered_airtel_data[delivered_airtel_data['presence_flag']==0]
print("Information missing for: ",len(missing_data), " cases")

Information missing for:  367  cases


In [7]:
# missing_data.to_excel('missing_data_' + curr_date + '.xlsx',index=False)

### Check 2: Consistency across reports - PD Dates (Point #4)

In [1]:
from os import listdir
from os.path import isfile, join

# Location of the CIR text files
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

pd_files = onlyfiles
pd_file_details = pd.DataFrame({'loan_ref_no':pd_files})

def date_calc(date):
    date = date.replace('PD - ','').replace('.xlsx','')
    return datetime.datetime.strptime(date,'%d%b%y').date()
pd_file_details['date'] = pd_file_details.apply(lambda x: date_calc(x['loan_ref_no']),axis=1)

pd_file_details = pd_file_details.sort_values('date', ascending = False)

master_pd_file = pd.DataFrame()

i=0

while i < len(pd_file_details):
    filename = pd_file_details.iloc[i,0]
    date = pd_file_details.iloc[i,1]
    file = pd.read_excel(mypath+"/"+filename)
    file = file[['MOBILE_NO','PD_DATE']]
    file = file.rename(columns={'PD_DATE': 'PD_DATE_'+str(date.strftime("%d%b%y"))})
    if len(master_pd_file) == 0:
        master_pd_file = file
    else:
        master_pd_file = pd.merge(master_pd_file, file, on = 'MOBILE_NO', how = 'outer')
    i+=1
    
new_columns = master_pd_file.columns.values

i=1
while i<len(new_columns):
    new_columns[i] = 'PD_'+str(i)
    i+=1
master_pd_file.columns = new_columns

def discrepancy(pd_1, pd_other):
    if pd_other is None or pd_other.lower() == 'nan' or pd_other.lower() == 'nat' or pd_other.lower() == '(blank)':
        return 0
    else:
        try:
            pd_1 = datetime.datetime.strptime(str(pd_1)[:10], '%Y-%m-%d')
            pd_other = datetime.datetime.strptime(str(pd_other)[:10], '%Y-%m-%d')
            if pd_1.date() == pd_other.date():
                return 0
            else:
                return 1
        except:
            return 1

i=2
while i < len(new_columns):
    master_pd_file['discrepancy_with_'+str(i)] = master_pd_file.apply(lambda x: discrepancy(x['PD_1'], str(x['PD_'+str(i)])), axis=1)
    if i==2:
        master_pd_file['discrepancy'] = master_pd_file['discrepancy_with_'+str(i)]
    else:
         master_pd_file['discrepancy'] = master_pd_file['discrepancy'] + master_pd_file['discrepancy_with_'+str(i)] 
    i+=1

master_pd_file[master_pd_file['discrepancy']>0]

In [9]:
# master_pd_file

### Check 3: PD Cases not reported on time (Point #2)

In [10]:
earlier_date = pd_file_details.iloc[1,1]

def pd_reporting(number, e_date, cur_date, disc):
    try:
        cur_date = datetime.datetime.strptime(str(cur_date)[:10],'%Y-%m-%d')
        if cur_date.date() < e_date and disc > 0:
            return 1
        else:
            return 0
    except:
        try:
            if cur_date is None or cur_date.lower() == 'nan' or cur_date.lower() == 'nat' or cur_date.lower() == '(blank)':
                return 0
        except:
            print("Something wrong with ",number)

master_pd_file['delay_in_reporting'] = master_pd_file.apply(lambda x: pd_reporting(x['MOBILE_NO'],earlier_date, str(x['PD_1']),x['discrepancy_with_2']),axis=1)
master_pd_file[master_pd_file['delay_in_reporting']==1]

,MOBILE_NO,PD_1,PD_2,discrepancy_with_2,discrepancy,delay_in_reporting


## EMI Recon for all Cases

In [2]:
pd_cases = delivered_airtel_data
pd_cases = pd_cases[['loan_ref_no','Loan Application Status','Device Delivered Date','Mobile_No','Airtel_Status']]

CLS_EMI_data = CLS_EMI_data[['Loan App Reference Number','Status','Lender: Account Name','Total Emi Paid Amount',
                             'Monthly Payment','First Monthly Collection Date','Delinquent Amount wrt MCD']]
CLS_EMI_data = CLS_EMI_data.rename(columns={'Loan App Reference Number': 'loan_ref_no'})

PD_CLS = pd.merge(pd_cases, CLS_EMI_data, on = 'loan_ref_no', how = 'left')

def date_calc_2(date):
    try:
        return datetime.datetime.strptime(date[:10],'%d/%m/%Y').date()
    except:
        try:
            return datetime.datetime.strptime(date[:10],'%Y-%m-%d').date()
        except:
            if date.lower() == 'nan' or date.lower() == 'nat':
                return 0
            else:
                return date 

filename = pd_file_details.iloc[0,0]
latest_pd = pd.read_excel(mypath+"/"+filename)
latest_pd = latest_pd[['MOBILE_NO', 'PD_DATE']]
latest_pd = latest_pd.rename(columns={'MOBILE_NO': 'Mobile_No'})
PD_CLS = pd.merge(PD_CLS, latest_pd, on = 'Mobile_No', how = 'left')
PD_CLS['Device Delivered Date'] = PD_CLS.apply(lambda x: date_calc_2(str(x['Device Delivered Date'])),axis=1)
PD_CLS['First Monthly Collection Date'] = PD_CLS.apply(lambda x: date_calc_2(str(x['First Monthly Collection Date'])),axis=1)
PD_CLS['Actual_PD_DATE'] = PD_CLS['PD_DATE']
PD_CLS['PD_DATE'] = PD_CLS['PD_DATE'].fillna(emi_analysis_date)
PD_CLS['PD_DATE'] = PD_CLS.apply(lambda x: date_calc_2(str(x['PD_DATE'])),axis=1)
PD_CLS = PD_CLS.fillna(0)
PD_CLS['EMIs_paid'] = PD_CLS['Total Emi Paid Amount'] / PD_CLS['Monthly Payment']

import dateutil.relativedelta as relativedelta

def first_emi(del_date):
    day_of_month = int(del_date.strftime("%d"))
    if day_of_month < 21:
        first_emi_date = del_date + relativedelta.relativedelta(months=1)
    else:
        first_emi_date = del_date + relativedelta.relativedelta(months=2)
    first_emi_date = first_emi_date.replace(day=5)
    return first_emi_date

PD_CLS['First_EMI_date_expected'] = PD_CLS.apply(lambda x: first_emi(x['Device Delivered Date']),axis=1)

def last_emi(pd_date):
    try:
        day_of_month = int(pd_date.strftime("%d"))
        if day_of_month < 5:
            pd_date_2 = pd_date - relativedelta.relativedelta(months=1)
        else:
            pd_date_2 = pd_date
        pd_date_2 = pd_date_2.replace(day=5)
        return pd_date_2
    except:
        return pd_date
    
PD_CLS['Last_EMI_date_expected'] = PD_CLS.apply(lambda x: last_emi(x['PD_DATE']),axis=1)

def expected_emi(loan, date_1, date_last):
    try:
        expected_emi = max((date_last.year - date_1.year) * 12 + date_last.month - date_1.month + 1,0)
    except Exception as e:
        print(loan, e)
        expected_emi = "Cannot be determined"
    return expected_emi

PD_CLS['Expected_EMI'] = PD_CLS.apply(lambda x: expected_emi(x['loan_ref_no'],x['First_EMI_date_expected'],x['Last_EMI_date_expected']),axis=1)

def diff_emi(exp, actual):
    try:
        return exp-actual
    except:
        return 'NA'

PD_CLS = PD_CLS.fillna(0)
    
PD_CLS['Pending EMIs'] = PD_CLS.apply(lambda x: diff_emi(x['Expected_EMI'], x['EMIs_paid']),axis=1)

PD_CLS['Pending Amount'] = PD_CLS.apply(lambda x: x['Pending EMIs'] * x['Monthly Payment'] if x['Pending EMIs'] != "NA" else "NA", axis=1)

#### Recon Sheet Output

In [12]:
ALL_CLS_closed = PD_CLS[PD_CLS['Status'] == 'Closed - Obligations met'].drop(['Pending Amount','Pending EMIs','Expected_EMI','Last_EMI_date_expected','First_EMI_date_expected','EMIs_paid','PD_DATE'],axis=1)
ALL_CLS_open = PD_CLS[PD_CLS['Status'] != 'Closed - Obligations met'].drop(['PD_DATE'],axis=1)

if not os.path.exists(report_folder+"/"+curr_mon):
    os.makedirs(report_folder+"/"+curr_mon)
writer = pd.ExcelWriter(report_folder+"/"+curr_mon + "/PD EMI Recon - All - "+curr_date+".xlsx")
master_pd_file.to_excel(writer,'PD File Issues',index = False)
missing_data.to_excel(writer,'Missing Data',index=False)
ALL_CLS_open.to_excel(writer,'Active Loans',index = False)
ALL_CLS_closed.to_excel(writer,'Pre-Closed Loans',index = False)
writer.save()

In [26]:
PD_CLS['Device Delivered Date'].dtype

dtype('O')